In [ ]:
#| default_exp web3

# Web3
> web3 wrapper

## Imports -

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from functools import update_wrapper
import inspect

from fastcore.all import custom_dir
from web3 import Web3, AsyncWeb3

In [ ]:
#| export
FIRST_POS_BLOCK = 15537394

## MultiRPCWeb3 -

In [ ]:
#| export
class MultiRPCWeb3:
    """Web3 object that tries to execute a method in multiple RPCs until one succeeds."""

    def __init__(self, *rpcs, providers=None):
        self.rpcs = rpcs
        self.providers = providers
        if not self.providers:
            self.providers = []
            for rpc in self.rpcs:
                if hasattr(rpc, 'provider'):
                    self.providers.append(rpc.provider)
        # hardcode genesis block timestamp (RPCs don't have it right)
        self.cache = {0: 1438269973}

    @classmethod 
    def from_rpcs(obj, *rpcs):
        return obj(*[Web3(Web3.HTTPProvider(rpc)) for rpc in rpcs])

    @classmethod 
    def async_from_rpcs(obj, *rpcs):
        return obj(*[AsyncWeb3(AsyncWeb3.AsyncHTTPProvider(rpc)) for rpc in rpcs])
    
    def __dir__(self): return custom_dir(self, add=self.rpcs[0].__dir__())
        
    def __getattr__(self, attr):

        def wrapper(*args, **kwargs):
            for i, (provider, rpc) in enumerate(zip(self.providers, self.rpcs)):
                try:
                    return rpc.__getattribute__(attr)(*args, **kwargs)
                except Exception as e:
                    print(f'RPC {provider} failed')
                    if i < len(self.providers) - 1:
                        print(f'Trying RPC {self.providers[i+1]}')
                    else:
                        print(f"All the RPCs failed to execute '{attr}'")
                        raise e

        async def awrapper(*args, **kwargs):
            for i, (provider, rpc) in enumerate(zip(self.providers, self.rpcs)):
                try:
                    return await rpc.__getattribute__(attr)(*args, **kwargs)
                except Exception as e:
                    print(f'RPC {provider} failed')
                    if i < len(self.providers) - 1:
                        print(f'Trying RPC {self.providers[i+1]}')
                    else:
                        print(f"All the RPCs failed to execute '{attr}'")
                        raise e

        for rpc in self.rpcs:
            rpc_attr = rpc.__getattribute__(attr)
            if callable(rpc_attr) and not inspect.iscoroutinefunction(rpc_attr):
                return update_wrapper(wrapper, rpc_attr)
            elif inspect.iscoroutinefunction(rpc_attr):
                return update_wrapper(awrapper, rpc_attr)
            elif attr in rpc.__dict__:
                return MultiRPCWeb3(*[rpc.__getattribute__(attr) for rpc in self.rpcs], providers=self.providers)
            elif attr in self.__dict__:
                return self.__getattribute__(attr)
        raise AttributeError(f"Attribute '{attr}' not found in any of the RPCs")

    def sort_providers(self, tolerance=0):
        """Sort provider by block number, so that the one with the highest block number is first."""
        rpcs = self.rpcs[:]
        providers = self.providers[:]
        last_block_number = []
        for i, (rpc, provider) in enumerate(zip(rpcs, providers)):
            self.rpcs = [rpc]
            self.providers = [provider]
            try:
                block_number = self.eth.get_block_number()
                last_block_number.append(block_number)
            except Exception as e:
                print(f'RPC {provider} failed')
                last_block_number.append(-1)
        max_block_number = max(last_block_number)
        for i in range(len(last_block_number)):
            if last_block_number[i] >= max_block_number - tolerance:
                last_block_number[i] = max_block_number
        index = sorted(range(len(last_block_number)), key=lambda k: last_block_number[k], reverse=True)
        self.rpcs = [rpcs[i] for i in index]
        self.providers = [providers[i] for i in index]


    def __getitem__(self, block_number):
        """The user can get the timestamp of a block by using the syntax `web3[block_number]`."""
        if block_number in self.cache:
            return self.cache[block_number]
        if block_number==-1:
            block_number = 'latest'
        block = self.eth.get_block(block_number)
        if block_number=='latest':
            block_number = block['number']
        self.cache[block_number] = block['timestamp']
        return block['timestamp']
    
    def __len__(self):
        return self.eth.get_block_number()

In [ ]:
show_doc(MultiRPCWeb3)

---

[source](https://github.com/flashbots/web3-data-tools/blob/main/web3_data_tools/web3.py#L17){target="_blank" style="float:right; font-size:smaller"}

### MultiRPCWeb3

>      MultiRPCWeb3 (*rpcs, providers=None)

Web3 object that tries to execute a method in multiple RPCs until one succeeds.

In [ ]:
show_doc(MultiRPCWeb3.sort_providers)

---

[source](https://github.com/flashbots/web3-data-tools/blob/main/web3_data_tools/web3.py#L79){target="_blank" style="float:right; font-size:smaller"}

### MultiRPCWeb3.sort_providers

>      MultiRPCWeb3.sort_providers (tolerance=0)

Sort provider by block number, so that the one with the highest block number is first.

## Example

In [ ]:
w3 = MultiRPCWeb3.async_from_rpcs('http://rpc1:1235', 'http://rpc2:2345')

In [ ]:
import os
RPC1 = os.environ['RPC1']
RPC2 = os.environ['RPC2']
w3 = MultiRPCWeb3.from_rpcs(RPC1, RPC2)

In [ ]:
w3.sort_providers(tolerance=5)

In [ ]:
len(w3)

18129311

In [ ]:
from web3_data_tools.core import interpolation_search
interpolation_search(w3, 1692548195, low=FIRST_POS_BLOCK)

17957091

## Export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()